In [133]:
!pip install mysql-connector-python

You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [134]:
import mysql.connector

In [135]:
from pprint import pprint

In [136]:
con = mysql.connector.connect(host='127.0.0.1', port=3306, database='writer_assistant', user='root', password='dbcourse')

In [137]:
cur = con.cursor(dictionary=True, buffered=True)

#### SELECT с несколькими (от трех) таблицами в условии выборки:
Кто из женщин, работающих в команде Хауса, был как-то связан с тюрьмой?

In [138]:
cur.execute("""SELECT full_name, occupations.name job, locations.name place_in_question, character2location.relation
               FROM characters
                 JOIN character2occupation
                 ON characters.id_character = character2occupation.char_id
                 JOIN occupations
                 ON character2occupation.occ_id = id_occupation
                 JOIN character2location
                 ON characters.id_character = character2location.char_id
                 JOIN locations
                 ON character2location.loc_id = locations.id_locations

               WHERE occupations.name = 'член команды диагностики'
               AND characters.gender = 'женщина'
               AND locations.name = 'Тюрьма'""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'full_name': 'Джессика Адамс',
  'job': 'член команды диагностики',
  'place_in_question': 'Тюрьма',
  'relation': 'работа'}]


,full_name,job,place_in_question,relation
0,Джессика Адамс,член команды диагностики,Тюрьма,работа


#### Вложенные SELECT:

Посмотрим, для каких персонажей есть записи об их отношениях с кем-то ещё

In [139]:
cur.execute("""SELECT full_name name
               FROM characters
               WHERE id_character IN (
                 SELECT char1_id
                 FROM character2character
               )""")
pprint(cur.fetchall())

[{'name': 'Грегори Хаус'},
 {'name': 'Лиза Кадди'},
 {'name': 'Джеймс Уилсон'},
 {'name': 'Эрик Форман'},
 {'name': 'Эллисон Кэмерон'},
 {'name': 'Роберт Чейз'},
 {'name': 'Реми Хэдли'},
 {'name': 'Эмбер Волакис'},
 {'name': 'Джессика Адамс'},
 {'name': 'Чи Пак'}]


Посмотрим, для кого мы знаем что-нибудь про жильё

In [140]:
cur.execute("""SELECT full_name name
               FROM characters
               WHERE id_character IN (
                   SELECT char_id FROM character2location
                   WHERE relation = 'жильё'
               )""")
pprint(cur.fetchall())

[{'name': 'Грегори Хаус'}, {'name': 'Джеймс Уилсон'}]


#### Запросы с JOIN:

Продолжая тему жилья, присоединим эту информацию к тем персонажам, для которых эта информация есть

In [141]:
cur.execute("""SELECT full_name, age, locations.name accomodation
               FROM characters
                 LEFT JOIN (
                     SELECT * FROM character2location
                     WHERE character2location.relation = 'жильё'
                     ) acc
                 ON id_character = acc.char_id
                 LEFT JOIN locations
                 ON acc.loc_id = id_locations""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'accomodation': 'Квартира Хауса', 'age': 52, 'full_name': 'Грегори Хаус'},
 {'accomodation': None, 'age': 43, 'full_name': 'Лиза Кадди'},
 {'accomodation': 'Квартира Хауса', 'age': 44, 'full_name': 'Джеймс Уилсон'},
 {'accomodation': None, 'age': 37, 'full_name': 'Эрик Форман'},
 {'accomodation': None, 'age': 31, 'full_name': 'Эллисон Кэмерон'},
 {'accomodation': None, 'age': 34, 'full_name': 'Роберт Чейз'},
 {'accomodation': None, 'age': 27, 'full_name': 'Реми Хэдли'},
 {'accomodation': None, 'age': 32, 'full_name': 'Эмбер Волакис'},
 {'accomodation': None, 'age': None, 'full_name': 'Джессика Адамс'},
 {'accomodation': None, 'age': 25, 'full_name': 'Марта М. Мастерс'},
 {'accomodation': None, 'age': None, 'full_name': 'Чи Пак'},
 {'accomodation': None, 'age': 46, 'full_name': 'Крис Тауб'},
 {'accomodation': None, 'age': 28, 'full_name': 'Лоренс Катнер'}]


,full_name,age,accomodation
0,Грегори Хаус,52.0,Квартира Хауса
1,Лиза Кадди,43.0,None
2,Джеймс Уилсон,44.0,Квартира Хауса
3,Эрик Форман,37.0,None
4,Эллисон Кэмерон,31.0,None
5,Роберт Чейз,34.0,None
6,Реми Хэдли,27.0,None
7,Эмбер Волакис,32.0,None
8,Джессика Адамс,NaN,None
9,Марта М. Мастерс,25.0,None


Узнаем, кто из персонажей работал в команде Хауса

In [142]:
cur.execute("""SELECT full_name, occupations.name job
               FROM characters
                 JOIN character2occupation
                 ON characters.id_character = character2occupation.char_id
                 JOIN occupations
                 ON character2occupation.occ_id = id_occupation
               WHERE occupations.name = 'член команды диагностики'""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'full_name': 'Эрик Форман', 'job': 'член команды диагностики'},
 {'full_name': 'Эллисон Кэмерон', 'job': 'член команды диагностики'},
 {'full_name': 'Роберт Чейз', 'job': 'член команды диагностики'},
 {'full_name': 'Реми Хэдли', 'job': 'член команды диагностики'},
 {'full_name': 'Джессика Адамс', 'job': 'член команды диагностики'},
 {'full_name': 'Чи Пак', 'job': 'член команды диагностики'},
 {'full_name': 'Крис Тауб', 'job': 'член команды диагностики'}]


,full_name,job
0,Эрик Форман,член команды диагностики
1,Эллисон Кэмерон,член команды диагностики
2,Роберт Чейз,член команды диагностики
3,Реми Хэдли,член команды диагностики
4,Джессика Адамс,член команды диагностики
5,Чи Пак,член команды диагностики
6,Крис Тауб,член команды диагностики


И кто был заведующим чего угодно

In [143]:
cur.execute("""SELECT full_name, occupations.name job
               FROM characters
                 JOIN character2occupation
                 ON characters.id_character = character2occupation.char_id
                 JOIN occupations
                 ON character2occupation.occ_id = id_occupation
               WHERE occupations.name LIKE 'заведующий%'""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'full_name': 'Грегори Хаус', 'job': 'заведующий отделением диагностики'},
 {'full_name': 'Эрик Форман', 'job': 'заведующий отделением диагностики'},
 {'full_name': 'Джеймс Уилсон', 'job': 'заведующий отделением онкологии'},
 {'full_name': 'Лиза Кадди', 'job': 'заведующий больницы'},
 {'full_name': 'Эрик Форман', 'job': 'заведующий больницы'}]


,full_name,job
0,Грегори Хаус,заведующий отделением диагностики
1,Эрик Форман,заведующий отделением диагностики
2,Джеймс Уилсон,заведующий отделением онкологии
3,Лиза Кадди,заведующий больницы
4,Эрик Форман,заведующий больницы


Посмотрим, с какими людьми связаны места кроме больницы (с ней и так связаны все, это сериал про больницу)

In [144]:
cur.execute("""SELECT locations.name place, full_name
               FROM locations
                 JOIN character2location
                 ON id_locations = loc_id
                 JOIN characters
                 ON char_id = id_character
               WHERE locations.name != 'Принстон-Плейнсборо'""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'full_name': 'Грегори Хаус', 'place': 'Мэйфилд'},
 {'full_name': 'Грегори Хаус', 'place': 'Тюрьма'},
 {'full_name': 'Джессика Адамс', 'place': 'Тюрьма'},
 {'full_name': 'Грегори Хаус', 'place': 'Квартира Хауса'},
 {'full_name': 'Джеймс Уилсон', 'place': 'Квартира Хауса'}]


,place,full_name
0,Мэйфилд,Грегори Хаус
1,Тюрьма,Грегори Хаус
2,Тюрьма,Джессика Адамс
3,Квартира Хауса,Грегори Хаус
4,Квартира Хауса,Джеймс Уилсон


#### Аггрегация и сортировки:

Отсортируем персонажей по количеству поклонников, то есть тех, кто влюблён в них

In [145]:
cur.execute("""SELECT full_name, COUNT(relation) n_admirers
               FROM character2character
                 INNER JOIN characters
                 ON char2_id = id_character
               WHERE relation = 'влюблённость'
               GROUP BY char2_id
               ORDER BY n_admirers DESC""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'full_name': 'Грегори Хаус', 'n_admirers': 2},
 {'full_name': 'Лиза Кадди', 'n_admirers': 1},
 {'full_name': 'Эрик Форман', 'n_admirers': 1},
 {'full_name': 'Реми Хэдли', 'n_admirers': 1},
 {'full_name': 'Эмбер Волакис', 'n_admirers': 1},
 {'full_name': 'Джеймс Уилсон', 'n_admirers': 1}]


,full_name,n_admirers
0,Грегори Хаус,2
1,Лиза Кадди,1
2,Эрик Форман,1
3,Реми Хэдли,1
4,Эмбер Волакис,1
5,Джеймс Уилсон,1


И посмотриим на самые популярные локации

In [146]:
cur.execute("""SELECT locations.name, locations.info, COUNT(char_id) n_connected
               FROM locations
                 INNER JOIN character2location
                 ON id_locations = loc_id
               GROUP BY locations.name
               ORDER BY n_connected DESC""")
d = cur.fetchall()
pprint(d)
pd.DataFrame(d)

[{'info': 'Больница находится в Принстоне, Нью-Джерси, США. Работает на '
          'благотворительные средства, является филиалом медицинского '
          'университета.',
  'n_connected': 13,
  'name': 'Принстон-Плейнсборо'},
 {'info': 'Хаус провёл там около года.', 'n_connected': 2, 'name': 'Тюрьма'},
 {'info': 'Адрес 221b Baker Street, как у Шерлока Холмса',
  'n_connected': 2,
  'name': 'Квартира Хауса'},
 {'info': 'Основана в 1876 году.', 'n_connected': 1, 'name': 'Мэйфилд'}]


,name,info,n_connected
0,Принстон-Плейнсборо,"Больница находится в Принстоне, Нью-Джерси, СШ...",13
1,Тюрьма,Хаус провёл там около года.,2
2,Квартира Хауса,"Адрес 221b Baker Street, как у Шерлока Холмса",2
3,Мэйфилд,Основана в 1876 году.,1


#### Ввод, модификация, удаление данных

* изменить данные о Эмбер, добавив в инфо тот факт, что Хаус никогда не называл её по фамилии
* добавить какого-то неканонного персонажа
* и удалить его

In [147]:
info = 'Она единственная среди подчинённых Хауса, кого он называл по имени'
cur.execute(f"""UPDATE characters SET backstory = '{info}'
                WHERE full_name = 'Эмбер Волакис'""")
con.commit()

# проверка
cur.execute('SELECT full_name, backstory FROM characters WHERE id_character = 8')
cur.fetchall()

[{'full_name': 'Эмбер Волакис',
  'backstory': 'Она единственная среди подчинённых Хауса, кого он называл по имени'}]

In [148]:
cur.execute("""INSERT INTO characters (short_name, age, role)
                VALUES ('Иван', '99', 'второстепенный')""")
con.commit()

# проверка
cur.execute("SELECT * FROM characters ORDER BY id_character DESC LIMIT 1")
pprint(cur.fetchall())

[{'age': 99,
  'aka': None,
  'backstory': None,
  'eye_color': None,
  'full_name': None,
  'gender': None,
  'hair_color': None,
  'id_character': 22,
  'race': None,
  'role': 'второстепенный',
  'short_name': 'Иван',
  'skin_color': None}]


In [149]:
cur.execute("DELETE FROM characters WHERE short_name = 'Иван'")
con.commit()

# проверка
cur.execute("SELECT * FROM characters ORDER BY id_character DESC LIMIT 1")
pprint(cur.fetchall())

[{'age': 28,
  'aka': None,
  'backstory': 'Его родители были застрелены, с 6 лет жил в приёмной семье.',
  'eye_color': 'карий',
  'full_name': 'Лоренс Катнер',
  'gender': 'мужчина',
  'hair_color': 'чёрный',
  'id_character': 13,
  'race': 'человек',
  'role': 'главный',
  'short_name': 'Катнер',
  'skin_color': 'тёмный'}]
